In [24]:
# Import Dependencies
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [29]:
def init_browser():
    executable_path = {'chromedriver':'../Documents/chromedriver'}
    return Browser('chrome', executable_path, headless=False)

In [32]:
def scrape_info():
    browser = init_broswer()
    final_dict = {}
    
    # Visit NASA Mars News Site 
    news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    browser.visit(news_url)
    html = browser.html
    news_soup = BeautifulSoup(html, "html.parser")
    
    # Scrape title and paragraph info
    news_title = news_soup.find_all('div', class_='content_title')[0].text
    news_p = news_soup.find_all('div', class_='article_teaser_body')[0].text
    
    # Visit the JPL Mars Space Site and scrap featured images
    jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    featured_img_url = ""
    
    
    
    
    
    
    final_dict = {'news_title': news_title
                 'news_p': news_p}
    
    return final_dict

SyntaxError: invalid syntax (<ipython-input-32-e86c123086b0>, line 18)